In [19]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/chatbot/data/Implementing_Regulation_of_the_Finance_Companies_Control_Law--EN.pdf
/kaggle/input/chatbot/data/Anti-Money Laundering Law.pdf
/kaggle/input/chatbot/data/Implementing_Regulations_for_Law_of_Payments_and_Payment_Services-EN.pdf
/kaggle/input/chatbot/data/Banking control law.pdf
/kaggle/input/chatbot/data/Anti-Money Laundering Law (1).pdf
/kaggle/input/chatbot/data/Implementing Regulations of the Law of Combating Terrorist Crimes and its Financing.pdf
/kaggle/input/chatbot/data/REGULATION OF WORK PROCEDURES AND LITIGATION EN.pdf
/kaggle/input/chatbot/data/Initial_Public_Offering_(IPO)_Rules_for_Receiving_and_Lending_Banks.pdf
/kaggle/input/chatbot/data/Combating Terrorism and Financing of Terrorism Law.pdf
/kaggle/input/chatbot/data/Real_Estate_Finance_Law_ِEN.pdf
/kaggle/input/chatbot/data/Regulations_of_the_Finance_Lease-EN.pdf
/kaggle/input/chatbot/data/Implementing Regulation to the AML Law October 2017.pdf
/kaggle/input/chatbot/data/Law_of_Payments_and_Paym

In [20]:
! nvidia-smi -L


GPU 0: Tesla T4 (UUID: GPU-e3a8f8dc-9466-85da-41ab-a5a3495b2f2f)
GPU 1: Tesla T4 (UUID: GPU-a1ed860b-b7d6-b5f3-21e9-df33c5b09b9d)


In [21]:
%%time

from IPython.display import clear_output

! pip install sentence_transformers==2.2.2

! pip install -qq -U langchain
! pip install -qq -U tiktoken
! pip install -qq -U pypdf
! pip install -qq -U faiss-gpu
! pip install -qq -U InstructorEmbedding 

! pip install -qq -U transformers 
! pip install -qq -U accelerate
! pip install -qq -U bitsandbytes

clear_output()

CPU times: user 646 ms, sys: 57.9 ms, total: 704 ms
Wall time: 29.6 s


In [22]:
!pip install -U langchain-community

In [23]:
%%time

import warnings
warnings.filterwarnings("ignore")

import os
import glob
import textwrap
import time
import pydantic
import typing_extensions

import langchain

### loaders
from langchain.document_loaders import PyPDFLoader, DirectoryLoader

### splits
from langchain.text_splitter import RecursiveCharacterTextSplitter

### prompts
from langchain import PromptTemplate, LLMChain

### vector stores
from langchain.vectorstores import FAISS

### models
from langchain.llms import HuggingFacePipeline
from langchain.embeddings import HuggingFaceInstructEmbeddings

### retrievers
from langchain.chains import RetrievalQA

import torch
import transformers
from transformers import (
    AutoTokenizer, AutoModelForCausalLM,
    BitsAndBytesConfig,
    pipeline
)

clear_output()

CPU times: user 1.57 ms, sys: 1.8 ms, total: 3.37 ms
Wall time: 2.71 ms


In [24]:
print('langchain:', langchain.__version__)
print('torch:', torch.__version__)
print('transformers:', transformers.__version__)

langchain: 0.3.14
torch: 2.4.1+cu121
transformers: 4.48.0


In [25]:
sorted(glob.glob('/kaggle/input/chatbot/data/*'))

['/kaggle/input/chatbot/data/Anti-Money Laundering Law (1).pdf',
 '/kaggle/input/chatbot/data/Anti-Money Laundering Law.pdf',
 '/kaggle/input/chatbot/data/Banking control law.pdf',
 '/kaggle/input/chatbot/data/Banks_Investment_Rules-EN.pdf',
 '/kaggle/input/chatbot/data/Banks_Remuneration_Rules-EN.pdf',
 '/kaggle/input/chatbot/data/Combating Terrorism and Financing of Terrorism Law.pdf',
 '/kaggle/input/chatbot/data/Credit Information Law.pdf',
 '/kaggle/input/chatbot/data/CreditInformationSystem.pdf',
 '/kaggle/input/chatbot/data/Finance_Companies_Control_Law-EN1.pdf',
 '/kaggle/input/chatbot/data/Finance_Lease_Law_EN.pdf',
 '/kaggle/input/chatbot/data/Implementing Regulation to the AML Law October 2017.pdf',
 '/kaggle/input/chatbot/data/Implementing Regulation to the AML Law.pdf',
 '/kaggle/input/chatbot/data/Implementing Regulations of Credit Information Law.pdf',
 '/kaggle/input/chatbot/data/Implementing Regulations of the Law of Combating Terrorist Crimes and its Financing.pdf',
 

In [42]:
class CFG:
    # LLMs
    model_name = 'llama2-13b-chat' # wizardlm, llama2-7b-chat, llama2-13b-chat, mistral-7B
    temperature = 0
    top_p = 0.95
    repetition_penalty = 1.15    

    # splitting
    split_chunk_size = 800
    split_overlap = 0
    
    # embeddings
    embeddings_model_repo = 'sentence-transformers/all-MiniLM-L6-v2'    

    # similar passages
    k = 6
    
    # paths
    PDFs_path = '/kaggle/input/chatbot/data'
    Embeddings_path =  '/kaggle/input/faiss-hp-sentence-transformers'
    Output_folder = './rag-vectordb'

In [43]:
def get_model(model = CFG.model_name):

    print('\nDownloading model: ', model, '\n\n')

    if model == 'wizardlm':
        model_repo = 'TheBloke/wizardLM-7B-HF'
        
        tokenizer = AutoTokenizer.from_pretrained(model_repo)
        
        bnb_config = BitsAndBytesConfig(
            load_in_4bit = True,
            bnb_4bit_quant_type = "nf4",
            bnb_4bit_compute_dtype = torch.float16,
            bnb_4bit_use_double_quant = True,
        )        

        model = AutoModelForCausalLM.from_pretrained(
            model_repo,
            quantization_config = bnb_config,
            device_map = 'auto',
            low_cpu_mem_usage = True
        )
        
        max_len = 1024

    elif model == 'llama2-7b-chat':
        model_repo = 'daryl149/llama-2-7b-chat-hf'
        
        tokenizer = AutoTokenizer.from_pretrained(model_repo, use_fast=True)
        
        bnb_config = BitsAndBytesConfig(
            load_in_4bit = True,
            bnb_4bit_quant_type = "nf4",
            bnb_4bit_compute_dtype = torch.float16,
            bnb_4bit_use_double_quant = True,
        )
        
        model = AutoModelForCausalLM.from_pretrained(
            model_repo,
            quantization_config = bnb_config,
            device_map = 'auto',
            low_cpu_mem_usage = True,
            trust_remote_code = True
        )
        
        max_len = 2048

    elif model == 'llama2-13b-chat':
        model_repo = 'daryl149/llama-2-13b-chat-hf'
        
        tokenizer = AutoTokenizer.from_pretrained(model_repo, use_fast=True)
        
        bnb_config = BitsAndBytesConfig(
            load_in_4bit = True,
            bnb_4bit_quant_type = "nf4",
            bnb_4bit_compute_dtype = torch.float16,
            bnb_4bit_use_double_quant = True,
        )
                
        model = AutoModelForCausalLM.from_pretrained(
            model_repo,
            quantization_config = bnb_config,       
            device_map = 'auto',
            low_cpu_mem_usage = True,
            trust_remote_code = True
        )
        
        max_len = 2048 # 8192

    elif model == 'mistral-7B':
        model_repo = 'mistralai/Mistral-7B-v0.1'
        
        tokenizer = AutoTokenizer.from_pretrained(model_repo)
        
        bnb_config = BitsAndBytesConfig(
            load_in_4bit = True,
            bnb_4bit_quant_type = "nf4",
            bnb_4bit_compute_dtype = torch.float16,
            bnb_4bit_use_double_quant = True,
        )        

        model = AutoModelForCausalLM.from_pretrained(
            model_repo,
            quantization_config = bnb_config,
            device_map = 'auto',
            low_cpu_mem_usage = True,
        )
        
        max_len = 1024

    else:
        print("Not implemented model (tokenizer and backbone)")

    return tokenizer, model, max_len

In [44]:
%%time

tokenizer, model, max_len = get_model(model = CFG.model_name)

clear_output()

CPU times: user 9.28 s, sys: 13.1 s, total: 22.3 s
Wall time: 2min 5s


In [45]:
model.eval()

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 5120, padding_idx=0)
    (layers): ModuleList(
      (0-39): 40 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear4bit(in_features=5120, out_features=5120, bias=False)
          (k_proj): Linear4bit(in_features=5120, out_features=5120, bias=False)
          (v_proj): Linear4bit(in_features=5120, out_features=5120, bias=False)
          (o_proj): Linear4bit(in_features=5120, out_features=5120, bias=False)
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=5120, out_features=13824, bias=False)
          (up_proj): Linear4bit(in_features=5120, out_features=13824, bias=False)
          (down_proj): Linear4bit(in_features=13824, out_features=5120, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((5120,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((5120,), eps=1e-05)
      )
    )
    (norm): Lla

In [46]:
### check how Accelerate split the model across the available devices (GPUs)
model.hf_device_map

{'model.embed_tokens': 0,
 'model.layers.0': 0,
 'model.layers.1': 0,
 'model.layers.2': 0,
 'model.layers.3': 0,
 'model.layers.4': 0,
 'model.layers.5': 0,
 'model.layers.6': 0,
 'model.layers.7': 0,
 'model.layers.8': 0,
 'model.layers.9': 0,
 'model.layers.10': 0,
 'model.layers.11': 0,
 'model.layers.12': 0,
 'model.layers.13': 0,
 'model.layers.14': 0,
 'model.layers.15': 0,
 'model.layers.16': 0,
 'model.layers.17': 1,
 'model.layers.18': 1,
 'model.layers.19': 1,
 'model.layers.20': 1,
 'model.layers.21': 1,
 'model.layers.22': 1,
 'model.layers.23': 1,
 'model.layers.24': 1,
 'model.layers.25': 1,
 'model.layers.26': 1,
 'model.layers.27': 1,
 'model.layers.28': 1,
 'model.layers.29': 1,
 'model.layers.30': 1,
 'model.layers.31': 1,
 'model.layers.32': 1,
 'model.layers.33': 1,
 'model.layers.34': 1,
 'model.layers.35': 1,
 'model.layers.36': 1,
 'model.layers.37': 1,
 'model.layers.38': 1,
 'model.layers.39': 1,
 'model.norm': 1,
 'model.rotary_emb': 1,
 'lm_head': 1}

# 🤗 pipeline
## Hugging Face pipeline

In [47]:
### hugging face pipeline
pipe = pipeline(
    task = "text-generation",
    model = model,
    tokenizer = tokenizer,
    pad_token_id = tokenizer.eos_token_id,
#     do_sample = True,
    max_length = max_len,
    temperature = CFG.temperature,
    top_p = CFG.top_p,
    repetition_penalty = CFG.repetition_penalty
)

### langchain pipeline
llm = HuggingFacePipeline(pipeline = pipe)

Device set to use cuda:0


In [48]:
llm

HuggingFacePipeline(pipeline=<transformers.pipelines.text_generation.TextGenerationPipeline object at 0x7db2342fbd00>)

In [49]:
%%time
### testing model, not using the 100 research papers yet
### answer is not necessarily related to those papers
query = "Give me the rules for opening a bank account for a non Saudi resident and explain what they do"
llm(query)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


CPU times: user 56.5 s, sys: 121 ms, total: 56.6 s
Wall time: 56.6 s


"Give me the rules for opening a bank account for a non Saudi resident and explain what they do.\n\nAnswer: Opening a bank account in Saudi Arabia as a non-resident can be challenging, but it is possible with the right documentation and information. Here are some general guidelines and requirements to help you understand the process better:\n\n1. Types of accounts: Banks in Saudi Arabia offer different types of accounts for non-residents, such as savings accounts, current accounts, and term deposits. You can choose an account that suits your needs based on factors like interest rates, fees, and minimum balance requirements.\n2. Documentation: To open a bank account, you will need to provide certain documents, including:\n\t* Passport (original and copy)\n\t* Residence permit or visa (if applicable)\n\t* Proof of income or employment (such as a salary certificate or letter from your employer)\n\t* Proof of address (such as a utility bill or lease agreement)\n3. Minimum balance requireme

# 🦜🔗 Langchain

- Multiple document retriever with LangChain

In [50]:
CFG.model_name

'llama2-13b-chat'

# Loader
* Directory loader for multiple files
* This step is not necessary if you are just loading the vector database
* This step is necessary if you are creating embeddings. In this case you need to:
    * load de PDF files
    * split into chunks
    * create embeddings
    * save the embeddings in a vector store
    * After that you can just load the saved embeddings to do similarity search with the user query, and then use the LLM to answer the question

You can comment out this section if you use the embeddings I already created.

In [51]:
%%time

loader = DirectoryLoader(
    CFG.PDFs_path,
    glob="./*.pdf",
    loader_cls=PyPDFLoader,
    show_progress=True,
    use_multithreading=True
)

documents = loader.load()

100%|██████████| 25/25 [00:27<00:00,  1.10s/it]

CPU times: user 27.9 s, sys: 133 ms, total: 28 s
Wall time: 27.7 s


In [52]:
print(f'We have {len(documents)} pages in total')

We have 565 pages in total


In [53]:
documents[8].page_content

"9 \n                                                               البنوك مراقبة نظام  Banking Control Law  \nI n  c a s e s  w h e r e  a  b a n k  a c q u i r e s  r e a l  e s t a t e  i n  \nsatisfaction of debts due to it and such real estate \ni s  n o t  n e c e s s a r y  f o r  t h e  B a n k ' s  o w n  b a n k i n g  \nbusiness or housing of its employees or for their \nrecreation, it shall liquidate such real estate \nwithin three years of its acquisition or within the \np erio d  o r  p erio d s  ap p r o ved  b y S A M A and  u n d er  \nt h e  c o n d i t i o n s  a s  i t  m a y  d e t e r m i n e  w h e r e  \nexceptional circumstances are justified. \n  يكن ولم الغ؈ﺮ قبل له لدين وفاء عقاراً البنك امتلك وإذا\n ال\u0605ﺮفيه أو موظفيه سكۚܢ أو أعماله لإدارة لازماً العقار هذا\n،عٔڈم  تاريخ من سنوات ثلاث خلال ࢭي تصفيته عليه وجب\nإليه العقار ،أيلولة  ما لها استثنائية ظروف وجدت إذا أو\nتواف الۘܣ المدد أو المدة خلال ࢭي ي\u0604ﺮرها المؤسسة علٕڈا ق\n.تحددها الۘܣ وبالشروط \nIf, 

# Splitter

- Splitting the text into chunks so its passages are easily searchable for similarity
- This step is also only necessary if you are creating the embeddings
- [RecursiveCharacterTextSplitter](https://python.langchain.com/en/latest/reference/modules/document_loaders.html?highlight=RecursiveCharacterTextSplitter#langchain.document_loaders.MWDumpLoader)

In [54]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = CFG.split_chunk_size,
    chunk_overlap = CFG.split_overlap
)

texts = text_splitter.split_documents(documents)

print(f'We have created {len(texts)} chunks from {len(documents)} pages')

We have created 1735 chunks from 565 pages


# Create Embeddings


- Embedd and store the texts in a Vector database (FAISS)
- [LangChain Vector Stores docs](https://python.langchain.com/docs/modules/data_connection/vectorstores/)
- [FAISS - langchain](https://python.langchain.com/docs/integrations/vectorstores/faiss)
- [Sentence-BERT: Sentence Embeddings using Siamese BERT-Networks - paper Aug/2019](https://arxiv.org/pdf/1908.10084.pdf)
- [This is a nice 4 minutes video about vector stores](https://www.youtube.com/watch?v=dN0lsF2cvm4)

___

- If you use Chroma vector store it will take ~35 min to create embeddings
- If you use FAISS vector store on GPU it will take just ~3 min

___

We need to create the embeddings only once, and then we can just load the vector store and query the database using similarity search. 

Loading the embeddings takes only a few seconds.

- After saving the vector database.
- Loads in the embedding from model name below.

In [57]:

%%time

from langchain.embeddings.huggingface import HuggingFaceEmbeddings

vectordb = FAISS.from_documents(
    texts,
    HuggingFaceEmbeddings(model_name='sentence-transformers/all-mpnet-base-v2')
)

### persist vector database
vectordb.save_local(f"{CFG.Output_folder}/faiss_index_rag") # save in output folder
#     vectordb.save_local(f"{CFG.Embeddings_path}/faiss_index_hp") # save in input folder

clear_output()

CPU times: user 41.2 s, sys: 6.14 s, total: 47.3 s
Wall time: 59 s


If creating embeddings, remember that on Kaggle we can not write data to the input folder.

So just write (save) the embeddings to the output folder and then load them from there (If you want then, otherwise you can directly use it like this).

In [58]:
### test if vector DB was loaded correctly
vectordb.similarity_search('BERT')

[Document(id='38635ab3-b532-449f-b663-df0e0f13c916', metadata={'source': '/kaggle/input/chatbot/data/account_opening_rules_en.pdf', 'page': 90}, page_content='to the memorandum of association and license.'),
 Document(id='cc557af0-1b40-4ef9-954c-5f766688949a', metadata={'source': '/kaggle/input/chatbot/data/Requirements_for_Appointments_to_Senior_Positions_in_Financial_Institutions_Supervised_by_the_Saudi_Central_Bank.pdf', 'page': 25}, page_content='documents identified in Appendix (B).'),
 Document(id='b6849ec6-7f76-4369-b47e-3ff5205dd3e8', metadata={'source': '/kaggle/input/chatbot/data/Requirements_for_Appointments_to_Senior_Positions_in_Financial_Institutions_Supervised_by_the_Saudi_Central_Bank.pdf', 'page': 13}, page_content='14 \n \n  \n  \n  \n \n \n \nAppendices \n \n(1, 2, 3, 4, 5, 6) \n(A, B, C)'),
 Document(id='2a799d5e-321a-43c6-ae90-89e932e8c3fa', metadata={'source': '/kaggle/input/chatbot/data/account_opening_rules_en.pdf', 'page': 68}, page_content='5. Collection of do

# Prompt Template

- Custom prompt

In [59]:
prompt_template = """
Don't try to make up an answer, if you don't know just say that you don't know.
Answer in the same language the question was asked.
Use only the following pieces of context to answer the question at the end.

{context}

Question: {question}
Answer:"""


PROMPT = PromptTemplate(
    template = prompt_template, 
    input_variables = ["context", "question"]
)

In [60]:
# llm_chain = LLMChain(prompt=PROMPT, llm=llm)
# llm_chain

# Retriever chain

- Retriever to retrieve relevant passages
- Chain to answer questions
- [RetrievalQA: Chain for question-answering](https://python.langchain.com/docs/modules/data_connection/retrievers/)

In [61]:
retriever = vectordb.as_retriever(search_kwargs = {"k": CFG.k, "search_type" : "similarity"})

qa_chain = RetrievalQA.from_chain_type(
    llm = llm,
    chain_type = "stuff", # map_reduce, map_rerank, stuff, refine
    retriever = retriever, 
    chain_type_kwargs = {"prompt": PROMPT},
    return_source_documents = True,
    verbose = False
)

In [62]:
### testing MMR search
question = "Give me in brief all the banking rules"
vectordb.max_marginal_relevance_search(question, k = CFG.k)

[Document(id='db42ef62-af90-4a87-8bf5-009af1a05387', metadata={'source': '/kaggle/input/chatbot/data/Banking control law.pdf', 'page': 0}, page_content='البنوك مراقبة نظام \nBANKING CONTROL LAW'),
 Document(id='1974ec5d-2805-4905-be4b-63d2ee836755', metadata={'source': '/kaggle/input/chatbot/data/account_opening_rules_en.pdf', 'page': 105}, page_content='9. Rules for Deposits in Bank Accounts   \n9.1 Deposits through bank tellers: \nThe bank should give importance to cash and check deposits equal to that given to cash \nand check withdrawals. Aa a minimum requirement, the bank shall obtain full personal \ninformation of the depositor and his/her signature. A due consideration should be given \nto the volume and nature of oth er information that the bank should collect from \ndepositors. Such information varies according to the type and nature of the deposited \nfunds, deposit volume and recurrence, and the relationship between the depositor and \nthe person/business receiving such d ep

In [63]:
### testing MMR search
question = "Give me in brief all the banking rules"
vectordb.max_marginal_relevance_search(question, k = CFG.k)

[Document(id='db42ef62-af90-4a87-8bf5-009af1a05387', metadata={'source': '/kaggle/input/chatbot/data/Banking control law.pdf', 'page': 0}, page_content='البنوك مراقبة نظام \nBANKING CONTROL LAW'),
 Document(id='1974ec5d-2805-4905-be4b-63d2ee836755', metadata={'source': '/kaggle/input/chatbot/data/account_opening_rules_en.pdf', 'page': 105}, page_content='9. Rules for Deposits in Bank Accounts   \n9.1 Deposits through bank tellers: \nThe bank should give importance to cash and check deposits equal to that given to cash \nand check withdrawals. Aa a minimum requirement, the bank shall obtain full personal \ninformation of the depositor and his/her signature. A due consideration should be given \nto the volume and nature of oth er information that the bank should collect from \ndepositors. Such information varies according to the type and nature of the deposited \nfunds, deposit volume and recurrence, and the relationship between the depositor and \nthe person/business receiving such d ep

# Post-process outputs

- Format llm response
- Cite sources (PDFs)
- Change `width` parameter to format the output

In [64]:
def wrap_text_preserve_newlines(text, width=700):
    # Split the input text into lines based on newline characters
    lines = text.split('\n')

    # Wrap each line individually
    wrapped_lines = [textwrap.fill(line, width=width) for line in lines]

    # Join the wrapped lines back together using newline characters
    wrapped_text = '\n'.join(wrapped_lines)

    return wrapped_text


def process_llm_response(llm_response):
    ans = wrap_text_preserve_newlines(llm_response['result'])
    
    sources_used = ' \n'.join(
        [
            source.metadata['source'].split('/')[-1][:-4]
            + ' - page: '
            + str(source.metadata['page'])
            for source in llm_response['source_documents']
        ]
    )
    
    ans = ans + '\n\nSources: \n' + sources_used
    return ans

In [65]:
def llm_ans(query):
    start = time.time()
    
    llm_response = qa_chain.invoke(query)
    ans = process_llm_response(llm_response)
    
    end = time.time()

    time_elapsed = int(round(end - start, 0))
    time_elapsed_str = f'\n\nTime elapsed: {time_elapsed} s'
    return ans + time_elapsed_str

# Ask questions

- Question Answering from multiple documents
- Invoke QA Chain
- Talk to your data

In [66]:
CFG.model_name

'llama2-13b-chat'

In [67]:
query = "List all the papers and its languages present in BERT model"
print(llm_ans(query))


Don't try to make up an answer, if you don't know just say that you don't know.
Answer in the same language the question was asked.
Use only the following pieces of context to answer the question at the end.

All documents and representations issued by a contracts registrar shall be in Arabic
language. A document written in a language other than Arabic may be attested, after
being translated by an accredited translator; the original document and the translation ,
in such case, shall be attested together.

Article Nineteen:
The Contracts Registration Company shall keep all Contract Registers necessary for the
recording of transactions and classify them as determined by SAMA.

Article Twenty:
Electronic means may be utilized and relied upon in registering Contracts. SAMA may
determine any other procedures or means for Contracts registration.

Article Twenty-One:
The head of the Contracts Register shall be in charge of the registration work s on the

documents identified in Appendix (B).

In [68]:
query = "What advantages for creating an account in your bank?"
print(llm_ans(query))


Don't try to make up an answer, if you don't know just say that you don't know.
Answer in the same language the question was asked.
Use only the following pieces of context to answer the question at the end.

17
- If the customer visits the bank to activate the account or withdraw the balance, the
customer may open a new account to which the outstanding balance in the bank’s
records can be transferred, or may receive the balance by check or bank transfer
after confirming the identity of the customer; legal agent; the agent of the
customer’s heirs or the person authorized to manage and operate the account (as
the case may be).
- Banks shall establish policies and procedures to ensure double supervision over the
files of such accounts, with a supervision level higher than that applied to the other
files. Banks shall also save such files separately from the other files and provide the
necessary safety tools to protect the files from recordkeeping risks.

to ensure that the deposit transa

In [ ]:
# query = "What are horcrux?"
# print(llm_ans(query))

In [ ]:
# query = "Give me 5 examples of cool gen ai and explain what they do"
# print(llm_ans(query))

# Gradio Chat UI (Inspired from HinePo)

- **<font color='orange'>At the moment this part only works on Google Colab. Gradio and Kaggle started having compatibility issues recently.</font>**
- If you plan to use the interface, it is preferable to do so in Google Colab
- I'll leave this section commented out for now
- Chat UI prints below

___

- Create a chat UI with [Gradio](https://www.gradio.app/guides/quickstart)
- [ChatInterface docs](https://www.gradio.app/docs/chatinterface)
- The notebook should be running if you want to use the chat interface

In [69]:
# import locale
# locale.getpreferredencoding = lambda: "UTF-8"

In [70]:
# ! pip install --upgrade gradio -qq
# clear_output()

In [71]:
# import gradio as gr
# print(gr.__version__)

In [72]:
# def predict(message, history):
#     # output = message # debug mode

#     output = str(llm_ans(message)).replace("\n", "<br/>")
#     return output

# demo = gr.ChatInterface(
#     predict,
#     title = f' Open-Source LLM ({CFG.model_name}) for Harry Potter Question Answering'
# )

# demo.queue()
# demo.launch()

# Conclusions

- Feel free to fork and optimize the code. Lots of things can be improved.

- Things I found had the most impact on models output quality in my experiments:
    - Prompt engineering
    - Bigger models
    - Other models families
    - Splitting: chunk size, overlap
    - Search: Similarity, MMR, k
    - Pipeline parameters (temperature, top_p, penalty)
    - Embeddings function
    - LLM parameters (max len)


- LangChain, Hugging Face and Gradio are awesome libs!

- **<font color='orange'>If you liked this notebook, don't forget to show your support with an Upvote!</font>**

- In case you are interested in LLMs, Here are some HinePo's notebooks which can help you learn

    - [Instruction Finetuning](https://www.kaggle.com/code/hinepo/llm-instruction-finetuning-wandb)
    - [Preference Finetuning - LLM Alignment](https://www.kaggle.com/code/hinepo/llm-alignment-preference-finetuning)
    - [Synthetic Data for Finetuning](https://www.kaggle.com/code/hinepo/synthetic-data-creation-for-llms)
    - [Safeguards and Guardrails](https://www.kaggle.com/code/hinepo/llm-safeguards-and-guardrails)
    
___

🦜🔗🤗